In [1]:
import json
import pandas as pd
from time import time
from connections import get_mongo_client
from my_utils import insert_into_mssql, load_my_columns, \
    get_numbers, map_dff_to_my_columns, max_number_find

In [3]:
CONFIG_FILE = "mongodb_to_sql_config.json"

In [5]:
with open(CONFIG_FILE, "r") as config_file:
        config = json.load(config_file)

In [6]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

In [7]:
for entry in config:
    print(f"Processing entry: {entry['name']}")
    nums_table = entry['nums_table']
    write_to_table = entry["write_table"]
    columns_file = entry["columns_file"]
    query_path = entry["query_path"]
    projection_path = entry["projection_path"]

Processing entry: Contingent Liabilities
Processing entry: Contracts
Processing entry: Claims WO Contracts
Processing entry: Client
Processing entry: Credit requests
Processing entry: Dynamics of Scoring
Processing entry: Scoring
Processing entry: Open Contracts
Processing entry: Open Contracts
Processing entry: Overview
Processing entry: Phones


In [12]:
columns = load_my_columns(columns_file)
max_num = max_number_find(nums_table)

In [13]:
query = {
        query_path: {'$exists': True},
        'number': {'$gt': max_num}
    }
projection = {
        'number': 1,
        projection_path: 1
    }

In [15]:
docs = task_collection.find(query, projection)

In [16]:
write_to_table

'bronze.katm_077_phones_test'

In [17]:
for doc in docs:
    rows = []
    _id = str(doc.get('_id'))
    number = doc.get('number')
    fields = doc.get('data', {}).get(query_path, [])
    if isinstance(fields, dict):
        fields = [fields]
    elif fields is None:
        fields = []

    for field in fields:
        row = {'_id': _id, 'number': number, **field}
        rows.append(row)

    if rows:
        df = pd.DataFrame(rows)
        final_df = map_dff_to_my_columns(df, columns)
        # insert_into_mssql(final_df, write_to_table)

In [24]:
doc

{'_id': ObjectId('6789eae0d94a33b38e16712b'),
 'number': 8590728,
 'data': {'katm_077': {'return': {'data': {'client': {'phones': {'phone': [{'phone_number': '9982549656',
         'phone_number_change': '2024-01-08'},
        {'phone_number': '998972549656',
         'phone_number_change': '2021-07-06'}]}}}}}}}

In [23]:
def resolve_nested_field(data, path):
    keys = path.split('.')
    for key in keys:
        if isinstance(data, dict):
            data = data.get(key, {})
        else:
            return None  # Path does not exist
    return data

In [25]:
resolve_nested_field(doc, query_path)

[{'phone_number': '9982549656', 'phone_number_change': '2024-01-08'},
 {'phone_number': '998972549656', 'phone_number_change': '2021-07-06'}]

In [28]:
fields = (doc.get('data', {})
                    .get('katm_077', {})
                    .get('return', {})
                    .get('data', {})
                    .get('client', {})
                    .get('phones', {})
                    .get('phone', [])
)

In [29]:
fields

[{'phone_number': '9982549656', 'phone_number_change': '2024-01-08'},
 {'phone_number': '998972549656', 'phone_number_change': '2021-07-06'}]